In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from torchvision import transforms
from torch.utils import data
from datetime import datetime
import itertools

from dataset.db_query import *
from dataset.dataset import SingleSeasonSingleLeagueTimeSlices, SingleSeasonSingleLeague
from dataset.util import pretty_print_match_ts, pretty_print_match
from dataset.train_valid_test_loader import *

import logging
logging.basicConfig(level=logging.DEBUG)


In [3]:
sqlpath = "../data/database.sqlite"


In [4]:
def collect_data_stats(data):
    num_home_wins = 0
    num_away_wins = 0
    num_draws = 0

    for x, y in data:
        if y[0] == 1:
            num_home_wins += 1
        if y[1] == 1: 
            num_draws += 1
        if y[2] == 1:
            num_away_wins += 1
    
    N = len(data)
    assert num_away_wins + num_draws + num_home_wins == N
    
    return num_home_wins / N, num_draws / N, num_away_wins / N

In [15]:
data = make_small_train_set(sqlpath, undersample_probs=(0.6, 1.0, 1.0))

INFO:root:got 1458 rows from db
INFO:root:got 183978 rows from db
INFO:root:got 306 rows (league: NET (id 13274), season: 2014/2015) from db
DEBUG:root:time for loading sql data 2.79663348197937
DEBUG:root:time for processing data 6.930705308914185


In [16]:
len(data)

246

In [17]:
collect_data_stats(data)

(0.3170731707317073, 0.2967479674796748, 0.3861788617886179)